<p>
<font size='5' face='Georgia, Arial'>IIC2115 - Programación como herramienta para la ingeniería</font><br>
<font size='1'>Basado en material de Karim Pichara y Christian Pieringer. Todos los derechos reservados.</font>
</p>

El polimorfismo se refiere a "la propiedad de enviar mensajes sintácticamente iguales a objetos de tipos distintos" ([Wikipedia](https://es.wikipedia.org/wiki/Polimorfismo_(inform%C3%A1tica), 2017). Básicamente es utilizar objetos de tipos distintos (instancias de distintas clases) con la misma _interfaz_. _Overriding_ (sobreescritura) y _overloading_ (sobrecarga) son dos maneras de hacer polimorfismo.

- Overriding: ocurre cuando se implementa un método en una subclase que "invalida" la implementación del mismo método en la super clase.
   
- Overloading: es la capacidad de definir un método con el mismo nombre pero con distinto número y tipo de argumentos. Es la capacidad de una función de ejecutar distintas acciones dependiendo del tipo y número de argumentos que recibe. 
  
Python no soporta *overloading* de manera explícita, se puede "simular" usando algunos parámetros con valores por defecto o número de argumentos variables, pero no se puede definir la función más de una vez con distintos tipos y números de argumentos y esperar que ambas definiciones sean consideradas por el programa. A pesar de que podría parecer lo contrario, en la práctica, esto no es una limitante.
   
#### Ejemplo

La clase `Variable` representa un conjunto de datos cualquiera, mientras que la subclase `Ingresos` contiene un método para calcular el valor "representante" (algo así como el promedio, mediana, moda, etc.). Ocurre lo mismo con las subclases `Comuna` y `Puesto`: Si los datos corresponden a ingresos, el representante es el promedio. Si los datos corresponden a la comuna, el representante es la comuna que más se repite. Finalmente, si los datos corresponden al puesto de trabajo, entonces el representante es el que tiene el puesto más alto según la jerarquía especificada en el diccionario "categorías".

In [ ]:
import numpy as np  #veremos numpy en detalle más adelante, por el momento sólo lo utilizaremos

class Variable:
    
    def __init__(self, data):
        self.data = np.array(data)

    def representante(self):
        pass


class Ingresos(Variable):
    
    def representante(self):
        return np.mean(self.data)


class Comuna(Variable):
    
    def representante(self):
        ind = np.argmax([np.sum(self.data == c) for c in self.data])  # el que mas se repite
        return self.data[ind]


class Puesto(Variable):
    
    categorias = {'Gerente': 1, 'SubGerente': 2, 'Analista': 3, 
                  'Alumno en Practica': 4} # class (or static) variable

    def representante(self):
        return self.data[np.argmin([Puesto.categorias[c] for c in self.data])]#la categoria mas alta acorde con el diccionario

In [ ]:
lista_pesos = Ingresos([50, 80, 90, 150, 45, 65, 78, 89, 59, 77, 90])
lista_comunas = Comuna(['Providencia', 'Macul' , 'LaReina' ,'Santiago', 'Providencia', 'PuenteAlto',
                        'Macul', 'Santiago', 'Santiago' ])
lista_puestos = Puesto(['SubGerente', 'Analista','SubGerente','Analista','Alumno en Practica',
                        'Alumno en Practica'])


print(lista_pesos.representante())
print(lista_comunas.representante())
print(lista_puestos.representante())

En este último ejemplo podemos apreciar claramente como es posible reutilizar de manera eficiente una misma interfaz, para entregar resultados distintos (que además son generados a partir de dominions distintos)

<h1> Overriding de operadores en Python </h1>

Existen muchos operadores en Python que funcionan para varias de las clases "built-in". Por ejemplo, el operador "+" puede sumar dos números, concatenar dos strings, mezclar dos listas, etc. dependiendo de la clase con la que estemos trabajando:

In [ ]:
a = [1,2,3,4]
b = [5,6,7,8]
print(a+b)
c = "Hola"
d = " Mundo"
print(c+d)

Nosotros también podemos personalizar el método `__add__` para que funcione en algún tipo de clase específica que necesitemos. Por ejemplo, supongamos una clase que representa un carro de compra:

In [ ]:
class Carro:
    '''
    Un carro de compras lo representaremos como un diccionario 
    donde el key es el nombre del producto y el value es la cantidad
    Ej: {'pan' : 3, 'leche' : 2, 'agua' : 6}
    '''
    
    def __init__(self, lista_productos):
        self.lista_productos = lista_productos

    def __add__(self, otro_carro):
        lista_sumada = self.lista_productos
        for p in otro_carro.lista_productos.keys():  # aquí vamos recorriendo los nombres de los productos
            if p in self.lista_productos.keys():
                lista_sumada.update({ p : otro_carro.lista_productos[p] + self.lista_productos[p]})  # aquí creo la nueva instancia con las cantidades sumada
            else:
                lista_sumada.update({ p : otro_carro.lista_productos[p]})
                
        return Carro(lista_sumada)
    
    def __repr__(self):
        return "\n".join("Producto: {} | Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())

In [ ]:
carro_1 = Carro({'pan' : 3, 'leche' : 2, 'agua' : 6})
carro_2 = Carro({'leche' : 5, 'bebida' : 2, 'cerveza' : 12})
carro_3 = carro_1 + carro_2
print(carro_3.lista_productos)

 El método `__repr__` nos permite generar un string que será usado a la hora de llamar a print de alguna instancia de Carro:

In [ ]:
print(carro_3)

También podríamos haber implementado el método `__str__` que cumple la misma función que el método `__repr__`. La principal diferencia es que `__repr__` debería contener todos los detalles necesarios para identificar bien al objeto, como para ser usado por alguien que implementará algo en el futuro y debe entender bien nuestro código. El método `__str__` está orientado a generar una impresión "human-readable", algo que se vea bien y se interprete bien en el contexto en particular, pero no necesariamente debe contener todos los detalles técnicos del objeto. En casos en que `__str__` está implementado, print usará el string generado por `__str__` para imprimir, pero cuando no esté `__str__` implementado, print usará el método `__repr__`.

In [ ]:
class Carro:
    ''' Un carro de compras lo representaremos como un diccionario 
        donde el key es el nombre del producto y el value es la cantidad
        Ej: {'pan' : 3, 'leche' : 2, 'agua' : 6}
    '''
    def __init__(self, lista_productos):
        self.lista_productos = lista_productos

    def __add__(self, otro_carro):
        lista_sumada = self.lista_productos
        for p in otro_carro.lista_productos.keys():#aquí vamos recorriendo los nombres de los productos
            if p in self.lista_productos.keys():
                lista_sumada.update({ p : otro_carro.lista_productos[p] + self.lista_productos[p]})#aquí creo la nueva instancia con las cantidades sumada
            else:
                lista_sumada.update({ p : otro_carro.lista_productos[p]})
                
        return Carro(lista_sumada)
    
    def __repr__(self):
        s = self.__doc__#esto retorna el string del comienzo de la clase, la documentación que la describe
        return s + "\n" + "\n".join("Producto: {} | Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())
    
    def __str__(self):
        return "\n".join("Producto: {} - Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())    

In [ ]:
carro_1 = Carro({'pan' : 3, 'leche' : 2, 'agua' : 6})
carro_2 = Carro({'leche' : 5, 'bebida' : 2, 'cerveza' : 12})
carro_3 = carro_1 + carro_2
print(carro_3)

Si comentamos el método `__str__`, print va a imprimir el string que retorna la función `__repr__`

In [ ]:
class Carro:
    ''' Un carro de compras lo representaremos como un diccionario 
        donde el key es el nombre del producto y el value es la cantidad
        Ej: {'pan' : 3, 'leche' : 2, 'agua' : 6}
    '''
    def __init__(self, lista_productos):
        self.lista_productos = lista_productos

    def __add__(self, otro_carro):
        lista_sumada = self.lista_productos
        for p in otro_carro.lista_productos.keys():#aquí vamos recorriendo los nombres de los productos
            if p in self.lista_productos.keys():
                lista_sumada.update({ p : otro_carro.lista_productos[p] + self.lista_productos[p]})#aquí creo la nueva instancia con las cantidades sumada
            else:
                lista_sumada.update({ p : otro_carro.lista_productos[p]})
                
        return Carro(lista_sumada)
    
    def __repr__(self):
        s = self.__doc__#esto retorna el string del comienzo de la clase, la documentación que la describe
        return s + "\n" + "\n".join("Producto: {} | Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())
    
#    def __str__(self):
#        return "\n".join("Producto: {} - Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())    

In [ ]:
carro_1 = Carro({'pan' : 3, 'leche' : 2, 'agua' : 6})
carro_2 = Carro({'leche' : 5, 'bebida' : 2, 'cerveza' : 12})
carro_3 = carro_1 + carro_2
print(carro_3)

De la misma forma podemos personalizar la mayoría de los operadores, por ejemplo, para personalizar el operador "menor que" (less than):

In [ ]:
class Punto: 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y
    
    def __lt__(self, otro_punto): 
        self_mag = (self.x ** 2) + (self.y ** 2) 
        otro_punto_mag = (otro_punto.x ** 2) + (otro_punto.y ** 2) 
        return self_mag < otro_punto_mag

p1 = Punto(2,4)
p2 = Punto(8,3)
print(p1 < p2)

# Duck Typing

> if it walks like a duck and quacks like a duck then it is a duck" 
(no importa el tipo de objeto si contiene la acción)
 
Duck typing es una característica de algunos lenguajes que hace que el polimorfismo explícito sea menos relevante, ya que el lenguaje por sí sólo es capaz de generar comportamiento polimórfico sin la necesidad de implementar el polimorfismo a través de la herencia. 

In [ ]:
class Pato:
    
    def gritar(self):
        print("Quack!")
        
    def caminar(self):
        print("caminando como un pato...")        
    
class Persona:
    
    def gritar(self):
        print("Ahhh!")
        
    def caminar(self):
        print("caminando como un humano...")        

        
def activar(pato): #esto en otro tipo de lenguaje obligaría a que pato sea de la clase "Pato", por lo tanto
    pato.gritar()  #la función activar no podría ser llamada con un argumento tipo "Persona"
    pato.caminar()


donald = Pato()
juan = Persona()
activar(donald)
activar(juan)